In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

from selenium import webdriver

options = webdriver.ChromeOptions()
#options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")  # sandbox 주요 요소 처리불가-> nosandbox 처리 가능

driver = webdriver.Chrome(options=options)

In [30]:
url='https://huggingface.co/papers'
driver.get(url)
time.sleep(0.2)

real_all_title=[]
real_all_news=[]
real_all_url=[]
real_all_author=[] 
real_all_stars=[]


# 전 날 페이지로 넘기기
for d in range(2):
    all_title=[]
    all_news=[]
    all_url=[]
    all_author=[] 
    all_stars=[]

        
    titles=driver.find_elements(By.TAG_NAME,'h3')# 제목
    stars=driver.find_elements(By.TAG_NAME,'a>div>div') # 좋아요

    for i in titles:
        all_title.append(i.text)
        real_all_title.append(i.text)
    all_title=[i for i in all_title if i]  # 빈칸제거

    for i in stars:
        all_stars.append(i.text)
        real_all_title.append(i.text)


    for n in all_title:

        print(n)
        driver.find_element(By.PARTIAL_LINK_TEXT,n).click() #페이지 들어가기
        time.sleep(1)
        
        nourl=driver.current_url#url 저장
        real_all_url.append(nourl)
        time.sleep(1)
        
        news=driver.find_element(By.TAG_NAME,'p') # 뉴스 본문 저장
        real_all_news.append(news.text)

    
        author=driver.find_element(By.CLASS_NAME,'whitespace-nowrap.underline') #저자 
        real_all_author.append(author.text)
            
        
        driver.back()

    # 잠깐 쉬고 전 페이지 가기
    time.sleep(1)
    driver.find_element(By.CLASS_NAME,'btn.gap-2').click() # 사이트 전 날



FreeInit: Bridging Initialization Gap in Video Diffusion Models
FreeControl: Training-Free Spatial Control of Any Text-to-Image Diffusion Model with Any Condition
DiffMorpher: Unleashing the Capability of Diffusion Models for Image Morphing
Rethinking Compression: Reduced Order Modelling of Latent Features in Large Language Models
Alignment for Honesty
Interfacing Foundation Models' Embeddings
CCM: Adding Conditional Controls to Text-to-Image Consistency Models
Fast Training of Diffusion Transformer with Extreme Masking for 3D Point Clouds Generation
VILA: On Pre-training for Visual Language Models
Honeybee: Locality-enhanced Projector for Multimodal LLM
Steering Llama 2 via Contrastive Activation Addition
Llama Guard: LLM-based Input-Output Safeguard for Human-AI Conversations
COLMAP-Free 3D Gaussian Splatting
How Well Does GPT-4V(ision) Adapt to Distribution Shifts? A Preliminary Investigation
PEEKABOO: Interactive Video Generation via Masked-Diffusion
"I Want It That Way": Enabling 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".whitespace-nowrap.underline"}
  (Session info: chrome=120.0.6099.71)


In [26]:
sumnews=[]
for t,n,u,a,s in zip(all_title,all_news,all_url,all_author,all_stars):
    singles={'title':t,'news':n,'url':u,'author':a,'like':s}
    sumnews.append(singles)

import pandas as pd
news_df=pd.DataFrame(sumnews)
news_df

""


In [25]:
news_df.to_excel('samtest.xlsx')